In [140]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [141]:
#setup
from sklearn.preprocessing import LabelEncoder
data = pd.read_csv(r"C:\Users\micha\Downloads\data.csv", encoding='Windows-1252')
data=data.rename(columns={'Unnamed: 0':'Timestamp'})
data=data.sort_values(by=['Timestamp','asset_id'])
print(data.columns)

Index(['Timestamp', 'asset_id', 'operational_mode', 's1', 's2', 's3'], dtype='object')


In [143]:
"""Exploratory Data Analysis and Preprocessing"""

from sklearn.preprocessing import LabelEncoder
from datetime import datetime

#encode the modes
encoder = LabelEncoder()
encoder.fit(['a','b','c','d','e','f'])
encoded_modes = encoder.transform(['a','b','c','d','e','f'])
data['operational_mode_encoded']=encoder.transform(data['operational_mode'])

#convert the timestamp to an atomic value
data['atomic_timestamp'] = pd.to_datetime(data['Timestamp']).apply(lambda x: x.timestamp())


hour_of_day = lambda x: x.hour
data['hour'] = data['Timestamp'].apply(lambda x: hour_of_day(pd.to_datetime(x)))
scale_hour = lambda x: (2 * np.pi * x) / 24
data['hour'] = data['hour'].apply(lambda x: scale_hour(x))



#for weather reasons, operational mode may be skewed by time of day and day of year
minute_of_day = lambda x: (x.hour * 60) + x.minute
data['minute'] = data['Timestamp'].apply(lambda x: minute_of_day(pd.to_datetime(x)))
scale_minute = lambda x: (2 * np.pi * x) / 1440
data['minute'] = data['minute'].apply(lambda x: scale_minute(x))

day_of_year = lambda x: (2 * np.pi * x) / 365
data['day'] = pd.DatetimeIndex(data['Timestamp']).dayofyear
data['day'] = data['day'].apply(lambda x: day_of_year(x))
print(data.head)

#handling missing values 
data=data.dropna() 

asset_values_mean={}
asset_values_std={}
"""
hour_values_mean={}
hour_values_std={}
"""
#modes={'a','b','c','d','e','f'}
for i in range(1,37):
    asset_values_mean[i] = []
    asset_values_std[i]= []
    
#find the average mean and standard deviation for each sensor value for evvery asset_id
for i in range(1,37):
    for j in range(1,4):
        if i not in asset_values_mean:
            #changed
            asset_values_mean[i]=[]
            asset_values_std[i]=[]
        asset_values_mean[i].append(("s{}".format(j),sensor_val(i,j,"mean")))
        asset_values_std[i].append(("s{}".format(j),sensor_val(i,j,"std")))
print(asset_values_mean)

"""
for i in range(0,25):
    for j in range(1,4):
        if i not in hour_values_mean:
            hour_values_mean[i]=[]
            hour_values_std[i]=[]
        hour_values_mean[i].append(("s{}".format(j),data[data['hour']==i]["s{}".format(j)].mean()))
        hour_values_std[i].append(("s{}".format(j),data[data['hour']==i]["s{}".format(j)].std()))
"""
#after examining this data, we can conccur that the sensor values will strongly stray away from average tendancies for a given machine, in other words
#machines can only be compared to each other

#next we will add a row that instead of storing a sensor value, stores the amount of standard deviations away from te mean that the sensor value is.
#this will give the neural network true context


#Here we add our standardized sensor values to the set based on asset_id and hour
for i in range(1,4):
    #data["s{}_deviation_hour".format(i)] = data.apply(lambda row: (hour_values_mean[int(24*(row['hour']//(2*np.pi)))][i-1][1]-row["s{}".format(i)])/hour_values_std[int(24*(row['hour']//(2*np.pi)))][i-1][1], axis=1)
    data["s{}_deviation".format(i)]=data.apply(lambda row: (asset_values_mean[row['asset_id']][i-1][1]-row["s{}".format(i)])/asset_values_std[row['asset_id']][i-1][1], axis=1)
print(data.head)


    
#here we will simply drop rows that have outliers, specified by 3 standard deviations away from the mean

i=0
drop=[]
for index, row in data.iterrows():
    for j in range(1,4):
        sensor="s{}".format(j)
        #if sensor_outlier(row['operational_mode'],j,row[sensor])==True:
        if sensor_outlier(row['asset_id'],j,row[sensor])==True:
            drop.append(i)
        
    i=i+1

data.drop(drop,axis=0,inplace=True)
data.reset_index()

#Note training, validation, and test data is split below




<bound method NDFrame.head of                   Timestamp  asset_id operational_mode    s1     s2        s3  \
941955  2016-01-01 00:00:00         1                b  0.69  10.80 -2.030000   
998324  2016-01-01 00:00:00         2                b -0.50   9.72  1.540000   
884484  2016-01-01 00:00:00         3                b  2.87  18.81 -0.890000   
690926  2016-01-01 00:00:00         4                b  1.04   9.00  0.690000   
451356  2016-01-01 00:00:00         5                b -0.95   5.01  1.648062   
...                     ...       ...              ...   ...    ...       ...   
108100  2016-07-15 14:55:00        20                f -1.31   9.96  0.570000   
108101  2016-07-15 15:00:00        20                f -1.31   9.92  0.650000   
108102  2016-07-15 15:05:00        20                f -1.31   9.89  0.690000   
108103  2016-07-15 15:10:00        20                f -1.31   9.89  0.690000   
108104  2016-07-15 15:15:00        20                f -1.27   9.89  0.650000  

"\n\ntrain_data = data.sample(frac=0.2, random_state=42)\nval_data = data.drop(train_data.index)\ntrain_data_X = train_data[['s1_deviation', 's2_deviation', 's3_deviation','asset_id','minute','day']]\nprint(train_data_X.shape)\nval_data_X = val_data[['s1_deviation', 's2_deviation', 's3_deviation','asset_id','minute','day']]\n"

In [ ]:


#statistical analysis

#Visualization 
data['asset_id'].hist(bins=36)


plt.xlabel('Machine') 
plt.ylabel('Frequency') 
plt.title('Machine Usage') 
plt.xlim(1,36)

#displaying the plot
plt.show()


#precision of the sensors used for each device
sensor_precision=[]
activity=[]

#data_dict=data.to_dict("records")



#show the frequency of operational modes
plt.title('Operational Modes') 
plt.xlabel('Modes') 
data['operational_mode'].hist(bins=36)
plt.show()

#create an array with each machine, and show how long each machine is in each mode, use a dictionary with tuples

times=operation_mode_times()
print(times)    



precision={}
#find the average variance that the sensors of a certain machine have
for i in range(1,37):
    p=average_variance(i)
    precision[i]=p

#input any asset you want to the below code. Note comment out if you want runtime to be much faster
#print(find_change_indicies(4))


'\n\n#statistical analysis\n\n#Visualization \ndata[\'asset_id\'].hist(bins=36)\n\n#add labels and title\nplt.xlabel(\'Machine\') \nplt.ylabel(\'Frequency\') \nplt.title(\'Machine Usage\') \nplt.xlim(1,36)\n\n#display the plot\nplt.show()\n\n\n#precision of the sensors used for each device\nsensor_precision=[]\nactivity=[]\n\n#data_dict=data.to_dict("records")\n\n\n\n#show the frequency of operational modes\nplt.title(\'Operational Modes\') \nplt.xlabel(\'Modes\') \ndata[\'operational_mode\'].hist(bins=36)\nplt.show()\n\n#create an array with each machine, and show how long each machine is in each mode, use a dictionary with tuples\n\n#times=operation_mode_times()\n#print(times)    \n\n\n\nprecision={}\n#find the average variance that the sensors of a certain machine have\n\n\nfor i in range(1,37):\n    p=average_variance(i)\n    precision[i]=p\n\n\n#print(find_change_indicies(4))\n\n#below we will statistical information on the sensor value for each sensor in each mode, specify the te

In [ ]:
#for statistical analyisis only, fine the amount of times that assets are in particular modes. Only calculated by amount of apearences
def operation_mode_times():
    grouped = data.groupby(['asset_id', 'operational_mode'])
    result_dict = {}
    for (asset_id, operational_mode), count in grouped.size().iteritems():
        if asset_id not in result_dict:
            result_dict[asset_id] = []
        result_dict[asset_id].append((operational_mode, count))
    return result_dict

In [ ]:
#find the average variance that the sensors of a certain machine have
def average_variance(asset_id):
    # Filter the data by asset_id
    asset_data = data[data['asset_id'] == asset_id]
    # Calculate the variance for each row
    variances = asset_data[['s1', 's2', 's3']].var(axis=1)
    # Calculate the average variance
    avg_variance = np.mean(variances)
    return avg_variance

In [ ]:
#simple function for scaleability and ease of access to calculate certain test statistics based on an asset id
def sensor_val(asset,sensor,statistic):
    s="s{}".format(sensor)
    relevent_data=data[data['asset_id']==asset]
    values=relevent_data[s].tolist()
    if statistic=="mean":
        return np.mean(values)
    if statistic=="std":
        return np.std(values)
    if statistic=="max":
        return np.max(values)
    if statistic=="min":
        return np.min(values)
    else:
        return None
     

In [ ]:
#return the indicies for which a given asset changes operational modes
def find_change_indicies(id):
    indices = []
    relevant_data = data[data['asset_id'] == id]
    j=0
    for i, row in relevant_data.iterrows():
        if j > 0 and j < len(relevant_data) - 1 and row['operational_mode'] != relevant_data.iloc[j-1]['operational_mode']:
            indices.append(data[(data['Timestamp']==row['Timestamp']) & (data['asset_id']==id)].index.item())
        j=j+1
    return indices

In [ ]:
#determine if a sensor value is an outlier with respect to other readings for its asset_id
def sensor_outlier(asset,sensor,value):
    mean=asset_values_mean[asset][sensor-1][1]
    std=asset_values_std[asset][sensor-1][1]
    maxv=mean+(3*std)
    minv=mean-(3*std)
    if value>maxv or value<minv:
        return True
    return False
    


In [ ]:
#training data, verification data
print(data.shape)
print(data.head)


(1079042, 14)
<bound method NDFrame.head of                   Timestamp  asset_id operational_mode    s1     s2        s3  \
941955  2016-01-01 00:00:00         1                b  0.69  10.80 -2.030000   
998324  2016-01-01 00:00:00         2                b -0.50   9.72  1.540000   
884484  2016-01-01 00:00:00         3                b  2.87  18.81 -0.890000   
690926  2016-01-01 00:00:00         4                b  1.04   9.00  0.690000   
451356  2016-01-01 00:00:00         5                b -0.95   5.01  1.648062   
...                     ...       ...              ...   ...    ...       ...   
108100  2016-07-15 14:55:00        20                f -1.31   9.96  0.570000   
108101  2016-07-15 15:00:00        20                f -1.31   9.92  0.650000   
108102  2016-07-15 15:05:00        20                f -1.31   9.89  0.690000   
108103  2016-07-15 15:10:00        20                f -1.31   9.89  0.690000   
108104  2016-07-15 15:15:00        20                f -1.27   9.

In [150]:


import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LSTM
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


train_data, val_test_data = train_test_split(data, test_size=0.4, random_state=42)
val_data, test_data = train_test_split(val_test_data, test_size=0.5, random_state=42)


#split into training and test data
train_data_X = train_data[['s1_deviation', 's2_deviation', 's3_deviation','asset_id','minute','day','hour']]
val_data_X = val_data[['s1_deviation', 's2_deviation', 's3_deviation','asset_id','minute','day','hour']]
test_data_X = test_data[['s1_deviation', 's2_deviation', 's3_deviation','asset_id','minute','day','hour']]


"""
train_data_X = train_data[['s1_deviation', 's2_deviation', 's3_deviation','asset_id','minute','day','hour','s1_deviation_hour','s2_deviation_hour','s3_deviation_hour']]
val_data_X = val_data[['s1_deviation', 's2_deviation', 's3_deviation','asset_id','minute','day','hour','s1_deviation_hour','s2_deviation_hour','s3_deviation_hour']]
test_data_X = test_data[['s1_deviation', 's2_deviation', 's3_deviation','asset_id','minute','day','hour','s1_deviation_hour','s2_deviation_hour','s3_deviation_hour']]
"""


#split to training and test data
train_data_Y = to_categorical(train_data['operational_mode_encoded'])
val_data_Y = to_categorical(val_data['operational_mode_encoded'])
test_data_Y = to_categorical(test_data['operational_mode_encoded'])

input_shape = (train_data_X.shape[1], 1)
print(input_shape)


inputs = Input(shape=input_shape)
print(inputs)

x = LSTM(64, activation='relu')(inputs)

outputs = Dense(6, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


history = model.fit(train_data_X, train_data_Y, epochs=10, validation_data=(val_data_X, val_data_Y))

test_loss, test_accuracy = model.evaluate(test_data_X, test_data_Y)

predictions = model.predict(test_data_X)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

#best loss=0.16
#best accuracy=0.925
#after 10 epochs




(7, 1)
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name='input_23'), name='input_23', description="created by layer 'input_23'")
Epoch 1/10
20233/20233 [==============================] - 219s 11ms/step - loss: 0.7634 - accuracy: 0.6867 - val_loss: 0.5496 - val_accuracy: 0.7668
Epoch 2/10
20233/20233 [==============================] - 224s 11ms/step - loss: 0.4250 - accuracy: 0.8177 - val_loss: 0.3203 - val_accuracy: 0.8589
Epoch 3/10
20233/20233 [==============================] - 226s 11ms/step - loss: 0.2786 - accuracy: 0.8773 - val_loss: 0.2412 - val_accuracy: 0.8893
Epoch 4/10
20233/20233 [==============================] - 185s 9ms/step - loss: 0.2139 - accuracy: 0.9033 - val_loss: 0.2104 - val_accuracy: 0.9042
Epoch 5/10
20233/20233 [==============================] - 202s 10ms/step - loss: 0.1870 - accuracy: 0.9145 - val_loss: 0.2129 - val_accuracy: 0.9048
Epoch 6/10
20233/20233 [==============================] - 212s 10ms/step - loss: 0.1728 - accuracy: 